In [ ]:
import os

import numpy as np
import pandas as pd
from mylibs.tools import Experiment

In [ ]:
%matplotlib qt

In [ ]:
# Collect data
M_but = 74.12
M_ocm = 418.4
M_peta = 298.29
M_dmag = 198.12

but_rho = 0.82668

PATH = "D:\WORKS\Diplom-work\Experiments\Viscosity"

def to_df(exp: Experiment):
    df = pd.DataFrame()
    k = 1
    df = exp.d.copy(deep=True).rename(columns={"x": "Temperature", "y": "D"})
    df["Viscosity"] = k * (df["Temperature"] + 273.15) / (df["D"] * 0.001)

    df["compound"] = exp.info["compound"]
    df["rho"] = exp.info["rho"]
    df["w_mass"] = exp.info["w"]
    df["w_vol"] = df["w_mass"] * df["rho"] / but_rho
    df["D0"] = exp.info["D0"]
    df["E"] = exp.info["E"]
    df['Kin_viscosity'] = exp.info['Viscosity']
    df = df[
        [
            "compound",
            "rho",
            "w_mass",
            "w_vol",
            "Temperature",
            "time",
            "Viscosity",
            'Kin_viscosity',

            "D",

            "D0",
            "E",
        ]
    ]
    return df


def collect(M_comp, comp_list):
    df = pd.concat(comp_list)
    df["x"] = (
        (df["w_mass"] / M_but)
        / (df["w_mass"] / M_but + (100 - df["w_mass"]) / M_comp)
        * 100
    )
    return df

# OCM
comp_list = []
for i in range(9):
    file = [f"{PATH}\OCM{i}\{f}" for f in os.listdir(f"{PATH}\OCM{i}") if ".hdf5" in f][
        0
    ]
    comp_list.append(to_df(Experiment.load_hdf5(file)))

ocm = collect(M_ocm, comp_list)

# PETA
comp_list = []
for i in range(5):
    file = [
        f"{PATH}\PETA{i}\{f}" for f in os.listdir(f"{PATH}\PETA{i}") if ".hdf5" in f
    ][0]
    comp_list.append(to_df(Experiment.load_hdf5(file)))
peta = collect(M_peta, comp_list)

# Butanol
butanol = to_df(Experiment.load_hdf5(f"{PATH}\BUTANOL\data_BUT_up.hdf5"))
butanol["x"] = 100
butanol["w_vol"] = 100
butanol["rho"] = but_rho

# DMAG
comp_list = []
for i in range(5):
    file = [
        f"{PATH}\DMAG{i}\{f}" for f in os.listdir(f"{PATH}\DMAG{i}") if ".hdf5" in f
    ][0]
    comp_list.append(to_df(Experiment.load_hdf5(file)))
dmag = collect(M_dmag, comp_list)

In [ ]:
# All data
all_df = pd.concat([ocm, peta, butanol, dmag]).reset_index(drop=True)
DF = all_df.groupby(by=["compound", "x", "Temperature"]).mean().reset_index()
DF.to_csv('Datas/Viscosity_all_data.csv')
DF

In [ ]:
# Simple data
datas = []
for _, group in DF.groupby(by=['compound', 'x']):
    for i0 in [15, 20, 25, 30, 40]:
        window_df = group[(i0 - 1 <= group['Temperature']) & (group['Temperature'] <= i0 + 1)]
        window_note = window_df[
            ['x', 'Temperature', 'rho', 'w_mass', 'w_vol', 'D', 'Viscosity', 'E', 'D0']
        ].mean()
        window_note['compound'] = group['compound'].iloc[0]
        window_note['Temperature'] = i0
        if any(window_note.isna()):
            continue
        datas.append(window_note)

DF2 = pd.concat(datas, axis=1).T
DF2.to_csv('Datas/Viscosity.csv')
DF2

In [ ]:
res = {}
for compound in DF["compound"].unique():
    df_comp = DF[DF["compound"] == compound]
    xs = df_comp["x"].unique()
    rhos = np.array([df_comp[df_comp["x"] == x]["rho"].mean() for x in xs]) * 1000
    print(f"{compound: >10}:", *xs.astype(int))
    print("*" * 10, *rhos.astype(int))
    res[compound] = list(zip(xs.astype(int), rhos.astype(int)))